In [1]:
# File processing
import sys

# Data processing
import numpy as np
from numpy import linalg as LAnumpy

# Data display 
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.legend_handler import HandlerLine2D

# Constants

In [2]:
CHROMOSOME_NUMBER = 14
CHROMOSOME_NUMBER = str(CHROMOSOME_NUMBER)
EMBEDDING_SIZE = 3

# Tool functions

In [3]:
def centralize_numpy(z):
    return z - np.mean(z, axis=0)

In [4]:
def normalize_numpy(z):
    
    norm = LAnumpy.norm(z, 2, axis=1)
    max_norm = np.max(norm, axis=0)
    if max_norm == 0:
        max_norm = 1
    
    return z / max_norm

In [5]:
def centralize_and_normalize_numpy(z):
    
    # Translate
    z = centralize_numpy(z)
    
    # Scale
    z = normalize_numpy(z)
    
    return z

In [6]:
def kabsch_superimposition_numpy(pred_structure, true_structure):
    
    # Centralize and normalize to unit ball
    pred_structure_unit_ball = centralize_and_normalize_numpy(pred_structure)
    true_structure_unit_ball = centralize_and_normalize_numpy(true_structure)
    
    # Rotation (solution for the constrained orthogonal Procrustes problem, subject to det(R) = 1)
    m = np.matmul(np.transpose(true_structure_unit_ball), pred_structure_unit_ball)
    u, s, vh = np.linalg.svd(m)
    
    d = np.sign(np.linalg.det(np.matmul(u, vh)))
    a = np.eye(EMBEDDING_SIZE)
    a[-1,-1] = d
    
    r = np.matmul(np.matmul(u, a), vh)
    
    pred_structure_unit_ball = np.transpose(np.matmul(r, np.transpose(pred_structure_unit_ball)))
    
    return pred_structure_unit_ball, true_structure_unit_ball

In [7]:
def kabsch_distance_numpy(pred_structure, true_structure):
    
    pred_structure_unit_ball, true_structure_unit_ball = kabsch_superimposition_numpy(pred_structure, true_structure)
    
    # Structure comparison
    d = np.mean(np.sum(np.square(pred_structure_unit_ball - true_structure_unit_ball), axis=1))
    
    return d

In [8]:
def extract_structure_from_pdb_file(file_name):
    
    structure = []
    f=open(file_name,'r')
    while 1:
        line=f.readline()
        coordinates = line[30:55].strip().split('.')
        
        if not line or not len(coordinates) > 3: 
            break
                    
        x = coordinates[0] + '.' + coordinates[1][:3]
        y = coordinates[1][3:] + '.' + coordinates[2][:3]
        z = coordinates[2][3:] + '.' + coordinates[3]
            
        x=float(x)
        y=float(y)
        z=float(z)
            
        structure.append([x, y, z])
    f.close
    
    return np.asarray(structure)

# Import structures

In [9]:
dmax_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/3DMax/chr' + CHROMOSOME_NUMBER + '.pdb')
# chromosome3d_structure = \
#     extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/Chromosome3D/chr' + CHROMOSOME_NUMBER + '.pdb')
# chromsde_structure = \
#     extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/ChromSDE/chr' + CHROMOSOME_NUMBER + '.pdb')
# gem_structure = \
#     extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/GEM/chr' + CHROMOSOME_NUMBER + '.pdb')
hsa_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/HSA/chr' + CHROMOSOME_NUMBER + '.pdb')
infmod3dgen_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/InfMod3DGen/chr' + CHROMOSOME_NUMBER + '.pdb')
lordg_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/LorDG/chr' + CHROMOSOME_NUMBER + '.pdb')
minimds_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/miniMDS/chr' + CHROMOSOME_NUMBER + '.pdb')
minimds_structure = np.vstack((np.vstack((minimds_structure, minimds_structure[-1])), minimds_structure[-1]))
mogen_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/MOGEN/chr' + CHROMOSOME_NUMBER + '.pdb')
# pastis_structure = \
#     extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/Pastis/chr' + CHROMOSOME_NUMBER + '.pdb')
shrec3d_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/ShRec3D/chr' + CHROMOSOME_NUMBER + '.pdb')
simba3d_structure = \
    extract_structure_from_pdb_file('GM12878_3DStructures/KR_100kb/SIMBA3D/chr' + CHROMOSOME_NUMBER + '.pdb')

In [10]:
dmax_structure = centralize_and_normalize_numpy(dmax_structure)
# chromosome3d_structure = centralize_and_normalize_numpy(chromosome3d_structure)
# chromsde_structure = centralize_and_normalize_numpy(chromsde_structure)

# gem_structure = centralize_and_normalize_numpy(gem_structure)
hsa_structure = centralize_and_normalize_numpy(hsa_structure)
infmod3dgen_structure = centralize_and_normalize_numpy(infmod3dgen_structure)

lordg_structure = centralize_and_normalize_numpy(lordg_structure)
minimds_structure = centralize_and_normalize_numpy(minimds_structure)
mogen_structure = centralize_and_normalize_numpy(mogen_structure)

# pastis_structure = centralize_and_normalize_numpy(pastis_structure)
shrec3d_structure = centralize_and_normalize_numpy(shrec3d_structure)
simba3d_structure = centralize_and_normalize_numpy(simba3d_structure)

In [11]:
print(dmax_structure.shape)
print(hsa_structure.shape)
print(infmod3dgen_structure.shape)
print(lordg_structure.shape)
print(minimds_structure.shape)
print(mogen_structure.shape)
print(shrec3d_structure.shape)
print(simba3d_structure.shape)

(883, 3)
(883, 3)
(883, 3)
(883, 3)
(883, 3)
(883, 3)
(883, 3)
(883, 3)


In [12]:
dmax_structure, _ = kabsch_superimposition_numpy(dmax_structure, minimds_structure)
# chromosome3d_structure, _ = kabsch_superimposition_numpy(chromosome3d_structure, minimds_structure)
# chromsde_structure, _ = kabsch_superimposition_numpy(chromsde_structure, minimds_structure)
# gem_structure, _ = kabsch_superimposition_numpy(gem_structure, minimds_structure)
hsa_structure, _ = kabsch_superimposition_numpy(hsa_structure, minimds_structure)
infmod3dgen_structure, _ = kabsch_superimposition_numpy(infmod3dgen_structure, minimds_structure)
lordg_structure, _ = kabsch_superimposition_numpy(lordg_structure, minimds_structure)
minimds_structure, _ = kabsch_superimposition_numpy(minimds_structure, minimds_structure)
mogen_structure, _ = kabsch_superimposition_numpy(mogen_structure, minimds_structure)
# pastis_structure, _ = kabsch_superimposition_numpy(pastis_structure, minimds_structure)
shrec3d_structure, _ = kabsch_superimposition_numpy(shrec3d_structure, minimds_structure)
simba3d_structure, _ = kabsch_superimposition_numpy(simba3d_structure, minimds_structure)

In [13]:
# structures = [dmax_structure, chromosome3d_structure, chromsde_structure, 
#              gem_structure, hsa_structure, infmod3dgen_structure, lordg_structure,
#              minimds_structure, mogen_structure, pastis_structure, shrec3d_structure, simba3d_structure]

# Visualization

In [30]:
test_structure = shrec3d_structure

In [31]:
# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'type': 'scatter3d'}]])

# adding surfaces to subplots.
fig.add_trace(
    go.Scatter3d(
    x=test_structure[:,0], y=test_structure[:,1], z=test_structure[:,2],
    marker=dict(
        size=4,
        color=np.asarray(range(len(test_structure[:,0]))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
),row=1, col=1)

fig.update_layout(
    height=1000,
    width=1000
)

fig.show()

In [16]:
test_structure = simba3d_structure

In [17]:
# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'type': 'scatter3d'}]])

# adding surfaces to subplots.
fig.add_trace(
    go.Scatter3d(
    x=test_structure[:,0], y=test_structure[:,1], z=test_structure[:,2],
    marker=dict(
        size=4,
        color=np.asarray(range(len(test_structure[:,0]))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
),row=1, col=1)

fig.update_layout(
    height=1000,
    width=1000
)

fig.show()